# Raspagem de Dados com Selenium

Raspar os dados do [ClimaTempo](https://www.climatempo.com.br/previsao-do-tempo/fim-de-semana/cidade/558/saopaulo-sp)

## Configuração do notebook

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from lib.database import Database
from selenium import webdriver
from datetime import datetime
import pandas as pd
import json
from sqlite3 import IntegrityError

DB_NAME = 'db.sqlite3'

# Funções auxiliares
def date_from_timestamp(timestamp):
    return timestamp.strftime('%Y-%m-%d')

def today():
    return datetime.today().strftime('%Y-%m-%d')

## Banco de Dados

As informações raspadas pelo robôzinho serão armazenadas em um banco de dados SQLITE que possui o seguinte modelo:

![db](imgs/db.png)

O robôzinho faz os seguintes passos:
1. Acessa os locais armazenados no banco de dados
1. Para cada local, acessa sua URL
1. Armazena a data de acesso, o local e a data da previsão
1. Armazena a temperatura prevista para cada horário

In [3]:
# Inicializa variável do banco de dados
db = Database(DB_NAME)

Você pode alterar quais locais o robô vai raspar!
É só modificar o arquivo _locais.csv_, adicionando o nome do local e sua URL.
A célula abaixo pega os lugares listados no CSV e os carrega no banco de dados.

In [4]:
with db as connection:
    with open('data/locais.csv') as file:
        db.load_locais(file)

## Web Scraping com Selenium

### Código de Scraping

- Primeiro, é preciso instalar o WebDriver para que o Selenium possa operar as automações em um navegador

- Para isso, será utilizada a biblioteca **WebDriver Manager**, como explicado no artigo


### Para pegar os dados:
1. Acessa a URL do local
1. Acha o botão de 15 dias e clica nele
1. Procura na página os elementos que contêm as informações
1. Pega os dados, processa e armazena no banco

- O atributo _data-infos_ contém um vetor com todas as horas dia: de 00h até as 23h. Esse vetor tem a seguinte forma

```
[
   {
       "date":"2022-08-31 00:00:00",
       "temperature": { "temperature":11 },
   },
     ...
   {
       "date":"2022-08-31 23:00:00",
           "temperature":{ "temperature":11 },
   }
]     
```

In [10]:
def scrape_location(driver, local):
    driver.get(local['url'])
    dt_acesso = today()
    
    # achando o botão para a previsão de 15 dias e clicando
    predictions_15_days_btn = driver.find_element(By.ID, 'Botao_barra_navegacao_15_dias')
    driver.execute_script('arguments[0].click();', predictions_15_days_btn)
    
    for elementIndex in range(1, 4):
        with db as connection:
            # Acha elemento que possui informações sobre as previsões
            elementId = f"wrapper-chart-{elementIndex}"
            element = driver.find_element(By.ID, elementId)
            
            # Converte as informações em JSON
            data = json.loads(element.get_attribute("data-infos"))
            
            # Pega qual é a data de previsão
            dt_previsao = data[0]['date'].split()[0]

            # Registrando um acesso para o dia e o local da previsão que está sendo feita
            try:
                id_acesso = db.registra_acesso(local['id'], dt_acesso, dt_previsao)
            except IntegrityError:
                print("Acesso já feito")
                continue
            
            # Para cada horário previsto, adiciona um objeto no banco de dados
            for predictin_to_hour in data:
                horario_previsao = predictin_to_hour['date'].split()[1]
                temperature_value = predictin_to_hour['temperature']['temperature']
                db.registra_previsao(id_acesso, horario_previsao, temperature_value)

In [6]:
# Carregando os locais
with db as connection:
    locais = db.get_locais()
    
# Instanciando o driver com Web Driver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for local in locais:
    scrape_location(driver, local)

driver.quit()

[WDM] - Downloading: 100%|█████████████████| 6.74M/6.74M [00:01<00:00, 5.71MB/s]


# Exportando o banco para CSV

In [7]:
def export_db_to_csv(path_to_query_file):
    with open(path_to_query_file) as file:
        query = file.read()
    
    with db as connection:
        db_df = pd.read_sql_query(query, db.con)
        db_df.to_csv('database.csv', index=False)
    return db_df


In [8]:
# Exporta informações das previsões de todos os horarios pra todos os locais
db_df = export_db_to_csv('queries/all_previsoes.sql')

for group, frame in db_df.groupby(by=['DT_PREVISAO', 'CIDADE', 'HORARIO']):
    tmps = frame['TEMPERATURA'].unique()
    qt_tmps = len(tmps)
    if qt_tmps > 1:
        print(f"{group} teve {tmps}")

('2022-08-30', 'São Paulo', '00:00:00') teve [10 11]
('2022-08-30', 'São Paulo', '01:00:00') teve [11 12]
('2022-08-30', 'São Paulo', '02:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '05:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '06:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '08:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '09:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '10:00:00') teve [13 14]
('2022-08-30', 'São Paulo', '17:00:00') teve [14 15]
('2022-08-30', 'São Paulo', '21:00:00') teve [10 11 13]
('2022-08-30', 'São Paulo', '22:00:00') teve [10 11 12]
('2022-08-30', 'São Paulo', '23:00:00') teve [10 11 12]
('2022-08-31', 'Salvador', '00:00:00') teve [21 22 20]
('2022-08-31', 'Salvador', '01:00:00') teve [21 20]
('2022-08-31', 'Salvador', '02:00:00') teve [20 21 19]
('2022-08-31', 'Salvador', '03:00:00') teve [21 19]
('2022-08-31', 'Salvador', '04:00:00') teve [21 19]
('2022-08-31', 'Salvador', '05:00:00') teve [21 19]
('2022-08-31', 'Salvador', '06:00:00'